# Azure AI Agents Orchestration with Semantic Kernel - Complete Tutorial

🚀 **Master the Art of Multi-Agent Orchestration!**

This tutorial shows you how to orchestrate multiple Azure AI agents working together using Semantic Kernel, both **sequentially** and **concurrently**!

## What You'll Learn:

1. **Sequential Orchestration** - Agents working one after another in a pipeline
2. **Concurrent Orchestration** - Multiple agents working simultaneously
3. **Agent Specialization** - Creating specialized agents with unique plugins
4. **Workflow Patterns** - Real-world orchestration scenarios
5. **Performance Optimization** - When to use concurrent vs sequential patterns

**Prerequisites:** You should have completed the basic Azure AI Agents and Plugins tutorials first!

---

## 🎭 The Power of Agent Orchestration

Imagine having a team of AI specialists working together on complex tasks!

### Single Agent Limitations:
- One agent handles everything
- Limited expertise in specific domains
- Potential context overload
- Slower processing for complex workflows

### Multi-Agent Orchestration Benefits:
- **Specialization**: Each agent excels in specific tasks 🎯
- **Parallel Processing**: Multiple agents work simultaneously ⚡
- **Better Results**: Domain experts produce higher quality outputs 📈
- **Scalability**: Add more agents as needed 🔄
- **Resilience**: If one agent fails, others continue 💪

### Orchestration Patterns:

- **Sequential (Pipeline)**: Agent A → Agent B → Agent C
- **Concurrent (Parallel)**: Agent A + Agent B + Agent C → Combine Results
- **Hybrid**: Mix of sequential and concurrent patterns

Let's build this step by step! 🛠️

## 📋 Setup and Prerequisites

Before we start orchestrating agents, let's make sure we have everything we need:

1. ✅ Environment variables set (`PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`)
2. ✅ Azure authentication configured
3. ✅ Required packages installed

We'll import all the necessary libraries and create our specialized plugins for each agent.

In [1]:
# Install required packages
# !pip install semantic-kernel azure-identity requests asyncio

# Import everything we need for orchestration
import os
import asyncio
import json
import time
import random
from datetime import datetime
from typing import Any, Dict, List, Annotated

from semantic_kernel.agents import Agent, ChatCompletionAgent, ConcurrentOrchestration, SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatMessageContent

print("🎭 Orchestration packages imported successfully!")
print("🎯 Ready to create specialized agent teams with Semantic Kernel!")

# Verify environment variables
required_vars = ['AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_DEPLOYMENT_NAME']
for var in required_vars:
    if var in os.environ:
        print(f"✅ {var} is set")
    else:
        print(f"❌ {var} is missing!")
        
print("\n🔧 Note: Make sure you have set up your Azure OpenAI credentials!")

🎭 Orchestration packages imported successfully!
🎯 Ready to create specialized agent teams with Semantic Kernel!
✅ AZURE_OPENAI_ENDPOINT is set
✅ AZURE_OPENAI_API_KEY is set
✅ AZURE_OPENAI_DEPLOYMENT_NAME is set

🔧 Note: Make sure you have set up your Azure OpenAI credentials!


## 🏗️ Creating Specialized Agent Plugins

Let's create three specialized agents with unique capabilities:

1. **🔍 Research Agent** - Information gathering and analysis
2. **📊 Analytics Agent** - Data processing and calculations
3. **📝 Content Agent** - Writing and formatting

Each agent will have specialized plugins that make them experts in their domain!

In [2]:
# 🔍 Research Agent Plugins
class ResearchPlugin:
    """Plugin for research and information gathering tasks."""
    
    @kernel_function(
        description="Researches information about a topic and provides key facts",
        name="research_topic"
    )
    def research_topic(
        self, 
        topic: Annotated[str, "The topic to research"]
    ) -> Annotated[str, "JSON string with research findings"]:
        """Simulates research on a topic."""
        # In a real implementation, this would call APIs, search databases, etc.
        research_data = {
            "topic": topic,
            "key_facts": [
                f"Key insight about {topic} - statistical trends",
                f"Recent developments in {topic} field",
                f"Expert opinions on {topic} future"
            ],
            "sources": ["Academic Papers", "Industry Reports", "Expert Interviews"],
            "confidence": 85,
            "research_time": datetime.now().isoformat()
        }
        print(f"🔍 Research completed for: {topic}")
        return json.dumps(research_data, indent=2)
    
    @kernel_function(
        description="Validates information sources and checks credibility",
        name="validate_sources"
    )
    def validate_sources(
        self,
        sources: Annotated[str, "List of sources to validate"]
    ) -> Annotated[str, "Validation results"]:
        """Validates information sources."""
        validation = {
            "validated_sources": sources.split(","),
            "credibility_score": random.randint(75, 95),
            "recommendations": "Sources appear credible and current"
        }
        print(f"🔍 Source validation completed")
        return json.dumps(validation)

# 📊 Analytics Agent Plugins
class AnalyticsPlugin:
    """Plugin for data analysis and calculations."""
    
    @kernel_function(
        description="Analyzes data and calculates statistics",
        name="analyze_data"
    )
    def analyze_data(
        self,
        data_description: Annotated[str, "Description of the data to analyze"]
    ) -> Annotated[str, "Analysis results with statistics"]:
        """Performs data analysis."""
        # Simulate complex data analysis
        analysis = {
            "data_analyzed": data_description,
            "metrics": {
                "mean": round(random.uniform(10, 100), 2),
                "median": round(random.uniform(10, 100), 2),
                "std_deviation": round(random.uniform(5, 20), 2),
                "sample_size": random.randint(100, 10000)
            },
            "trends": ["Upward trend", "Seasonal variation", "Growth acceleration"],
            "insights": f"Key patterns identified in {data_description}",
            "analysis_timestamp": datetime.now().isoformat()
        }
        print(f"📊 Data analysis completed for: {data_description}")
        return json.dumps(analysis, indent=2)
    
    @kernel_function(
        description="Creates projections and forecasts based on data",
        name="create_forecast"
    )
    def create_forecast(
        self,
        baseline_data: Annotated[str, "Baseline data for forecasting"]
    ) -> Annotated[str, "Forecast results"]:
        """Creates forecasts based on data."""
        forecast = {
            "forecast_period": "Next 6 months",
            "projected_values": [round(random.uniform(80, 120), 1) for _ in range(6)],
            "confidence_interval": "85-95%",
            "risk_factors": ["Market volatility", "Seasonal effects"],
            "recommendations": "Monitor key indicators weekly"
        }
        print(f"📊 Forecast created based on: {baseline_data}")
        return json.dumps(forecast)

# 📝 Content Agent Plugins
class ContentPlugin:
    """Plugin for content creation and formatting."""
    
    @kernel_function(
        description="Creates well-formatted reports from research and analysis data",
        name="create_report"
    )
    def create_report(
        self,
        research_data: Annotated[str, "Research findings to include"],
        analysis_data: Annotated[str, "Analysis results to include"]
    ) -> Annotated[str, "Formatted report"]:
        """Creates a formatted report from data."""
        report = f"""
# Executive Summary Report

**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Research Findings
{research_data}

## Data Analysis
{analysis_data}

## Recommendations
- Implement data-driven strategies
- Monitor key performance indicators
- Schedule regular review cycles

---
*Report generated by AI Agent Orchestration System*
"""
        print(f"📝 Report created successfully")
        return report
    
    @kernel_function(
        description="Summarizes complex information into key points",
        name="create_summary"
    )
    def create_summary(
        self,
        content: Annotated[str, "Content to summarize"]
    ) -> Annotated[str, "Executive summary"]:
        """Creates an executive summary."""
        summary = {
            "executive_summary": "Key insights and recommendations from comprehensive analysis",
            "main_points": [
                "Research validates current market position",
                "Analytics reveal positive growth trends", 
                "Strategic opportunities identified"
            ],
            "action_items": [
                "Continue monitoring metrics",
                "Implement recommended changes",
                "Schedule follow-up analysis"
            ]
        }
        print(f"📝 Summary created")
        return json.dumps(summary, indent=2)

# Create plugin instances
research_plugin = ResearchPlugin()
analytics_plugin = AnalyticsPlugin()
content_plugin = ContentPlugin()

print("✅ All specialized plugins created!")
print("🔍 Research Plugin: Information gathering & validation")
print("📊 Analytics Plugin: Data analysis & forecasting")
print("📝 Content Plugin: Report creation & summarization")

✅ All specialized plugins created!
🔍 Research Plugin: Information gathering & validation
📊 Analytics Plugin: Data analysis & forecasting
📝 Content Plugin: Report creation & summarization


## 🤖 Creating Our Specialized Agent Team

Now let's create three specialized agents, each with their own expertise and plugins:

1. **Dr. Research** 🔍 - The information specialist
2. **Prof. Analytics** 📊 - The data scientist  
3. **Ms. Content** 📝 - The communication expert

Each agent will have distinct instructions and specialized plugins to excel in their domain!

In [7]:
def create_specialized_agents():
    """Create three specialized agents with unique capabilities using Semantic Kernel."""
    
    # Create the Azure OpenAI chat completion service
    chat_service = AzureChatCompletion(
        deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
        endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY")
    )
    
    # 🔍 Create Research Agent
    research_agent = ChatCompletionAgent(
        name="Dr_Research",
        instructions="""You are Dr. Research, a world-class information specialist and fact-checker.
        
        Your expertise:
        - Gathering comprehensive information on any topic
        - Validating sources and checking credibility
        - Providing well-researched, factual content
        - Identifying key insights and trends
        
        Always use your research plugins when asked to investigate topics.
        Be thorough, accurate, and cite your methodology.
        Focus on providing high-quality, verified information.
        Be VERY concise in your answers.""",
        service=chat_service,
        plugins=[research_plugin]
    )
    
    # 📊 Create Analytics Agent  
    analytics_agent = ChatCompletionAgent(
        name="Prof_Analytics",
        instructions="""You are Prof. Analytics, a brilliant data scientist and statistician.
        
        Your expertise:
        - Analyzing complex datasets and patterns
        - Creating statistical models and forecasts
        - Identifying trends and correlations
        - Providing data-driven insights and recommendations
        
        Always use your analytics plugins when working with data.
        Be precise with numbers and explain your analytical methodology.
        Focus on extracting actionable insights from information.
        Be VERY concise in your answers.""",
        service=chat_service,
        plugins=[analytics_plugin]
    )
    
    # 📝 Create Content Agent
    content_agent = ChatCompletionAgent(
        name="Ms_Content",
        instructions="""You are Ms. Content, an expert content creator and communication specialist.
        
        Your expertise:
        - Creating clear, engaging, and well-structured content
        - Synthesizing complex information into readable formats
        - Writing reports, summaries, and presentations
        - Ensuring consistent tone and professional formatting
        
        Always use your content plugins when creating reports or summaries.
        Focus on clarity, readability, and professional presentation.
        Make complex information accessible to any audience.
        Be VERY concise in your answers.""",
        service=chat_service,
        plugins=[content_plugin]
    )
    
    print("🎭 Specialized agent team created!")
    print(f"🔍 Dr. Research: Information gathering specialist")
    print(f"📊 Prof. Analytics: Data analysis expert")
    print(f"📝 Ms. Content: Communication specialist")
    
    return {
        'research': research_agent,
        'analytics': analytics_agent,
        'content': content_agent
    }

print("✅ Specialized agent creation function ready!")

✅ Specialized agent creation function ready!


## 🔄 Sequential Orchestration: Pipeline Pattern

In sequential orchestration, agents work one after another in a pipeline:

**Research Agent** → **Analytics Agent** → **Content Agent** → **Final Result**

This pattern is perfect when:
- Each agent depends on the previous agent's output
- You need a step-by-step workflow
- Quality control at each stage is important

Let's implement a sequential workflow for creating a comprehensive market analysis report!

In [ ]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Callback function to monitor agent responses during orchestration."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"\n[{timestamp}] 🤖 Agent: {message.name}")
    print(f"📝 Response: {message.content[:200]}..." if len(str(message.content)) > 200 else f"📝 Response: {message.content}")
    print("─" * 60)

async def run_sequential_orchestration(topic: str):
    """Demonstrates sequential agent orchestration using Semantic Kernel's SequentialOrchestration."""
    
    print(f"🔄 Starting Sequential Orchestration for: {topic}")
    print(f"{'='*80}")
    
    # Create our specialized agent team
    agents = create_specialized_agents()
    
    # Create the sequential orchestration with our agents
    # The agents will be executed in the order they are provided
    orchestration = SequentialOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']],
        agent_response_callback=agent_response_callback
    )
    
    # Create and start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print(f"\n🎯 Sequential Pipeline: Research → Analytics → Content")
        print(f"{'─'*60}")
        
        # Create the task that will flow through the pipeline
        task = f"""Please analyze the topic '{topic}' comprehensively:
        
        Research Agent: Gather comprehensive information, key facts, and recent developments about {topic}.
        Analytics Agent: Analyze the research data, identify trends, and create projections.
        Content Agent: Synthesize all findings into a well-formatted executive report.
        
        Each agent should build upon the previous agent's work to create a comprehensive analysis.
        Be very concise in your responses, focusing on key insights and actionable recommendations."""
        
        # Invoke the sequential orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Wait for the results
        final_result = await orchestration_result.get(timeout=1200)
        
        print(f"\n✅ Sequential Orchestration Complete!")
        print(f"🎯 Final Result: Comprehensive {topic} analysis through sequential pipeline")
        print(f"\n📄 Final Report:\n{final_result}")
        
        return final_result
        
    finally:
        # Stop the runtime
        await runtime.stop_when_idle()
        print(f"\n🧹 Sequential orchestration cleanup complete.")

print("✅ Sequential orchestration function ready!")

✅ Sequential orchestration function ready!


In [12]:
# Test Sequential Orchestration
sequential_topic = "Artificial Intelligence in Healthcare"

print(f"🚀 Testing Sequential Orchestration")
print(f"📋 Topic: {sequential_topic}")
print(f"⏱️ This will take a few moments as each agent works in sequence...")

start_time = time.time()
sequential_result = await run_sequential_orchestration(sequential_topic)
end_time = time.time()

print(f"\n⏱️ Sequential orchestration completed in {end_time - start_time:.2f} seconds")
print(f"\n🎉 Sequential Orchestration Success!")
print(f"📄 Generated comprehensive analysis with research, analytics, and professional formatting")

🚀 Testing Sequential Orchestration
📋 Topic: Artificial Intelligence in Healthcare
⏱️ This will take a few moments as each agent works in sequence...
🔄 Starting Sequential Orchestration for: Artificial Intelligence in Healthcare
🎭 Specialized agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

🎯 Sequential Pipeline: Research → Analytics → Content
────────────────────────────────────────────────────────────
🔍 Research completed for: Artificial Intelligence in Healthcare

[10:37:44] 🤖 Agent: Dr_Research
📝 Response: Executive Multiphase Brief: Artificial Intelligence in Healthcare  
(Outputs are nested: each phase builds on the prior one)

────────────────────────────────────────
1. Research Agent – Curated Fact-...
────────────────────────────────────────────────────────────

[10:37:49] 🤖 Agent: Prof_Analytics
📝 Response: Let me know if you’d like deeper analysis on any specific sub-topic (e.

## ⚡ Concurrent Orchestration: Parallel Pattern

In concurrent orchestration, multiple agents work simultaneously:

**Research Agent** ⟷ **Analytics Agent** ⟷ **Content Agent** → **Combine Results**

This pattern is perfect when:
- Agents can work independently on different aspects
- Speed is important (parallel processing)
- Tasks don't depend on each other's outputs

Let's implement concurrent orchestration for rapid multi-perspective analysis!

In [ ]:
async def run_concurrent_orchestration(topic: str):
    """Demonstrates concurrent agent orchestration using Semantic Kernel's ConcurrentOrchestration."""
    
    print(f"⚡ Starting Concurrent Orchestration for: {topic}")
    print(f"{'='*80}")
    
    # Create our specialized agent team
    agents = create_specialized_agents()
    
    # Create the concurrent orchestration with our agents
    # All agents will work on the same task simultaneously
    orchestration = ConcurrentOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']]
    )
    
    # Create and start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print(f"\n🎯 All agents working simultaneously on: {topic}")
        print(f"{'─'*60}")
        
        # Create the task for all agents to work on concurrently
        task = f"""Analyze '{topic}' from your specialized perspective:
        
        Research Agent: Focus on current trends, recent innovations, and market dynamics in {topic}.
        Analytics Agent: Analyze market data, growth patterns, and create projections for {topic}.
        Content Agent: Create an engaging framework and structure for a {topic} report.
        
        Each agent should work independently and provide insights from your area of expertise.
        Br very concise in your responses, focusing on key insights and actionable recommendations."""
        
        # Invoke the concurrent orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Wait for all agents to complete their tasks
        results = await orchestration_result.get(timeout=1200)
        
        print(f"\n🎉 All agents completed their tasks!")
        print(f"{'─'*60}")
        
        # Display individual results
        print(f"\n📊 Concurrent Analysis Results:")
        for i, result in enumerate(results, 1):
            result_preview = str(result.content)[:200] + "..." if len(str(result.content)) > 200 else str(result.content)
            print(f"\n   🤖 {result.name}:")
            print(f"   📝 {result_preview}")
        
        # Combine results for final output
        combined_result = f"""
# Concurrent Analysis Results for {topic}

## Multi-Agent Parallel Analysis
Generated by concurrent orchestration of specialized AI agents

{chr(10).join([f"### {result.name} Insights:{chr(10)}{result.content}{chr(10)}" for result in results])}

---
*Generated through concurrent agent orchestration using Semantic Kernel*
"""
        
        print(f"\n✅ Concurrent Orchestration Complete!")
        print(f"🎯 Final Result: Multi-perspective {topic} analysis from parallel agent work")
        
        return combined_result
        
    finally:
        # Stop the runtime
        await runtime.stop_when_idle()
        print(f"\n🧹 Concurrent orchestration cleanup complete.")

print("✅ Concurrent orchestration function ready!")

✅ Concurrent orchestration function ready!


In [14]:
# Test Concurrent Orchestration
concurrent_topic = "Renewable Energy Technologies"

print(f"⚡ Testing Concurrent Orchestration")
print(f"📋 Topic: {concurrent_topic}")
print(f"🚀 All agents will work simultaneously for maximum speed!")

start_time = time.time()
concurrent_result = await run_concurrent_orchestration(concurrent_topic)
end_time = time.time()

print(f"\n⏱️ Concurrent orchestration completed in {end_time - start_time:.2f} seconds")
print(f"\n🎉 Concurrent Orchestration Success!")
print(f"📄 Generated multi-perspective analysis through parallel agent collaboration")

⚡ Testing Concurrent Orchestration
📋 Topic: Renewable Energy Technologies
🚀 All agents will work simultaneously for maximum speed!
⚡ Starting Concurrent Orchestration for: Renewable Energy Technologies
🎭 Specialized agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

🎯 All agents working simultaneously on: Renewable Energy Technologies
────────────────────────────────────────────────────────────
🔍 Research completed for: Renewable Energy Technologies
🔍 Source validation completed
📊 Data analysis completed for: Global renewable energy market 2013-2023: Installed capacity grew from 1,700 GW to 3,500 GW (CAGR ≈ 7.2%). Annual investments rose from $300 B (2013) to $495 B (2022). Segment shares 2023: Solar 46 %, Wind 34 %, Hydro 15 %, Others 5 %.
📊 Forecast created based on: Installed capacity 3,500 GW 2023; forecast CAGR 6% next 5 yrs; investment $495B 2022, expected to grow 8% CAGR.
📝 Report 

## 🔄 vs ⚡ Performance Comparison

Let's compare the performance and characteristics of sequential vs concurrent orchestration:

### 📊 Performance Analysis:

| Pattern | Speed | Quality | Use Case |
|---------|-------|---------|----------|
| **Sequential** 🔄 | Slower | Higher | Pipeline workflows, dependent tasks |
| **Concurrent** ⚡ | Faster | Different | Independent tasks, parallel processing |

Let's run a direct comparison to see the differences!

In [16]:
async def compare_orchestration_patterns():
    """Compare sequential vs concurrent orchestration patterns."""
    
    comparison_topic = "Smart Cities and IoT Integration"
    
    print(f"🏁 ORCHESTRATION PATTERN COMPARISON")
    print(f"{'='*80}")
    print(f"📋 Topic: {comparison_topic}")
    print(f"🎯 Goal: Compare speed, approach, and results of both patterns")
    
    # Test Sequential Pattern
    print(f"\n🔄 Testing Sequential Pattern...")
    seq_start = time.time()
    
    try:
        sequential_result = await run_sequential_orchestration(comparison_topic)
        seq_end = time.time()
        seq_time = seq_end - seq_start
        seq_success = True
    except Exception as e:
        seq_end = time.time()
        seq_time = seq_end - seq_start
        seq_success = False
        print(f"❌ Sequential pattern error: {e}")
    
    # Small delay between tests
    await asyncio.sleep(2)
    
    # Test Concurrent Pattern
    print(f"\n⚡ Testing Concurrent Pattern...")
    conc_start = time.time()
    
    try:
        concurrent_result = await run_concurrent_orchestration(comparison_topic)
        conc_end = time.time()
        conc_time = conc_end - conc_start
        conc_success = True
    except Exception as e:
        conc_end = time.time()
        conc_time = conc_end - conc_start
        conc_success = False
        print(f"❌ Concurrent pattern error: {e}")
    
    # Performance Analysis
    print(f"\n📊 PERFORMANCE COMPARISON RESULTS")
    print(f"{'='*80}")
    
    print(f"\n🔄 Sequential Orchestration:")
    print(f"   ⏱️  Time: {seq_time:.2f} seconds")
    print(f"   ✅  Success: {seq_success}")
    print(f"   🎯  Approach: Step-by-step pipeline with agent dependencies")
    
    print(f"\n⚡ Concurrent Orchestration:")
    print(f"   ⏱️  Time: {conc_time:.2f} seconds")
    print(f"   ✅  Success: {conc_success}")
    print(f"   🎯  Approach: Parallel processing with independent agents")
    
    if seq_success and conc_success:
        speed_improvement = ((seq_time - conc_time) / seq_time) * 100
        print(f"\n📈 Speed Improvement: {speed_improvement:.1f}% faster with concurrent pattern")
        
        if speed_improvement > 0:
            print(f"🏆 Concurrent orchestration was faster!")
        else:
            print(f"🐌 Sequential orchestration was faster (unusual but possible)")
    
    print(f"\n💡 KEY INSIGHTS:")
    print(f"   🔄 Sequential: Better for dependent tasks, higher context quality")
    print(f"   ⚡ Concurrent: Better for independent tasks, faster execution")
    print(f"   🎯 Choose based on your specific workflow requirements!")

print("✅ Orchestration comparison function ready!")

await compare_orchestration_patterns()

✅ Orchestration comparison function ready!
🏁 ORCHESTRATION PATTERN COMPARISON
📋 Topic: Smart Cities and IoT Integration
🎯 Goal: Compare speed, approach, and results of both patterns

🔄 Testing Sequential Pattern...
🔄 Starting Sequential Orchestration for: Smart Cities and IoT Integration
🎭 Specialized agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

🎯 Sequential Pipeline: Research → Analytics → Content
────────────────────────────────────────────────────────────
🔍 Research completed for: Smart Cities and IoT Integration
🔍 Source validation completed

[10:41:10] 🤖 Agent: Dr_Research
📝 Response: EXECUTIVE REPORT  
Topic: Smart Cities & IoT Integration  
Date: 4 Jun 2025  

────────────────────────────────────────  
1. RESEARCH AGENT – Key Facts & Recent Developments  
• Global market: USD ≈ 6...
────────────────────────────────────────────────────────────
📊 Forecast created based on: Glob

## 🎯 Real-World Orchestration Scenarios

Let's explore practical scenarios where you'd use each orchestration pattern:

### 🔄 Sequential Orchestration Use Cases:

1. **Document Processing Pipeline**
   - OCR Agent → Text Analysis Agent → Summary Agent
   
2. **Financial Analysis Workflow**
   - Data Collection Agent → Risk Analysis Agent → Report Generation Agent
   
3. **Software Development Pipeline**
   - Code Analysis Agent → Testing Agent → Documentation Agent

### ⚡ Concurrent Orchestration Use Cases:

1. **Multi-Source Research**
   - News Agent + Academic Agent + Social Media Agent → Combine Insights
   
2. **Customer Service Routing**
   - Intent Agent + Sentiment Agent + Priority Agent → Route to Human
   
3. **Content Generation**
   - Text Agent + Image Agent + SEO Agent → Multi-media Content


## 🏆 Best Practices for Agent Orchestration

### ✅ Design Principles:

1. **Agent Specialization**
   - Give each agent a clear, focused role
   - Create specialized plugins for domain expertise
   - Write specific instructions for each agent

2. **Error Handling**
   - Always implement proper cleanup (delete agents/threads)
   - Handle agent failures gracefully
   - Provide fallback strategies

3. **Performance Optimization**
   - Use concurrent patterns for independent tasks
   - Use sequential patterns for dependent workflows
   - Monitor execution times and optimize bottlenecks

4. **Resource Management**
   - Clean up threads and agents after use
   - Monitor Azure AI service usage and costs
   - Implement timeouts for long-running operations

### 🔒 Security Considerations:

1. **Data Privacy**
   - Don't log sensitive information
   - Implement data retention policies
   - Use secure communication between agents

2. **Access Control**
   - Limit agent permissions appropriately
   - Validate all inputs to plugins
   - Monitor agent activities

### 📊 Monitoring and Debugging:

1. **Logging**
   - Log agent interactions and decisions
   - Track execution times and performance metrics
   - Monitor plugin function calls

2. **Testing**
   - Test individual agents before orchestration
   - Test orchestration patterns with various scenarios
   - Implement automated testing for critical workflows

## 🚀 Advanced Semantic Kernel Orchestration Features

Semantic Kernel provides powerful additional features for orchestration beyond basic sequential and concurrent patterns:

### 🔧 Advanced Capabilities:

1. **Cancellation Support** 🛑
   - Cancel orchestrations that are taking too long
   - Graceful handling of cancelled operations
   - Timeout management

2. **Structured Outputs** 📋  
   - Get results in predefined data structures
   - Type-safe orchestration results
   - JSON schema validation

3. **Agent Response Callbacks** 👂
   - Monitor agent interactions in real-time
   - Log and debug orchestration flows
   - Custom response processing

4. **Runtime Management** ⚙️
   - InProcessRuntime for local orchestration
   - Efficient resource management
   - Automatic cleanup

Let's explore these advanced features!

In [ ]:
async def demonstrate_advanced_features():
    """Demonstrate advanced Semantic Kernel orchestration features."""
    
    print("🚀 Advanced Orchestration Features Demo")
    print("="*50)
    
    # Create agents for demonstration
    agents = create_specialized_agents()
    
    # 1. Demonstrate Agent Response Callback with Enhanced Monitoring
    print("\n👂 1. Enhanced Agent Response Monitoring Demo")
    print("-" * 40)
    
    def detailed_callback(message: ChatMessageContent) -> None:
        """Enhanced callback with comprehensive details."""
        timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]  # Include milliseconds
        print(f"\n[{timestamp}] 🤖 Agent Response:")
        print(f"    📛 Name: {message.name}")
        print(f"    📝 Content Length: {len(message.content)} characters")
        print(f"    🔤 Content Preview: {message.content[:150]}{'...' if len(str(message.content)) > 150 else ''}")
        if hasattr(message, 'metadata') and message.metadata:
            print(f"    📊 Metadata: {message.metadata}")
        print("    " + "─" * 50)
    
    # Create a sequential orchestration with detailed monitoring
    monitored_orchestration = SequentialOrchestration(
        members=[agents['research']],  # Just one agent for demo
        agent_response_callback=detailed_callback
    )
    
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print("🎯 Running monitored orchestration...")
        result = await monitored_orchestration.invoke(
            task="Provide a brief overview of renewable energy trends focusing on solar power adoption",
            runtime=runtime
        )
        
        final_result = await result.get(timeout=1200)
        print(f"\n✅ Monitored orchestration completed!")
        print(f"📄 Final Result Length: {len(str(final_result))} characters")
        
    except Exception as e:
        print(f"❌ Error in monitored orchestration: {e}")
        
    finally:
        await runtime.stop_when_idle()
    
    # 2. Demonstrate Timeout and Error Handling
    print("\n⏰ 2. Timeout and Error Handling Demo")
    print("-" * 40)
    
    quick_orchestration = SequentialOrchestration(
        members=[agents['analytics'], agents['content']],
        agent_response_callback=agent_response_callback
    )
    
    runtime2 = InProcessRuntime()
    runtime2.start()
    
    try:
        print("🎯 Testing timeout handling (using very short timeout for demo)...")
        result = await quick_orchestration.invoke(
            task="Analyze quantum computing market potential and create executive summary",
            runtime=runtime2
        )
        
        # Use a very short timeout to demonstrate timeout handling
        try:
            final_result = await result.get(timeout=2)  # Very short timeout for demo
            print(f"✅ Completed within timeout!")
        except asyncio.TimeoutError:
            print(f"⏰ Operation timed out as expected (demo timeout was very short)")
        except Exception as e:
            print(f"❌ Other error: {type(e).__name__}: {e}")
            
    finally:
        await runtime2.stop_when_idle()
    
    # 3. Demonstrate Concurrent with Multiple Specialized Tasks
    print("\n⚡ 3. Advanced Concurrent Orchestration Demo")
    print("-" * 40)
    
    concurrent_orchestration = ConcurrentOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']]
    )
    
    runtime3 = InProcessRuntime()
    runtime3.start()
    
    try:
        print("🎯 Running advanced concurrent orchestration...")
        
        advanced_task = """Analyze 'Artificial Intelligence in Education' with each agent focusing on different aspects:
        
        Research Agent: Focus on current AI educational tools, case studies, and adoption rates in schools/universities.
        Analytics Agent: Analyze the effectiveness data, ROI metrics, and performance improvements from AI in education.
        Content Agent: Create a framework for an implementation guide for educational institutions considering AI adoption.
        
        Provide comprehensive insights from your specialized perspective."""
        
        result = await concurrent_orchestration.invoke(
            task=advanced_task,
            runtime=runtime3
        )
        
        results = await result.get(timeout=1200)
        
        print(f"\n🎉 Advanced concurrent orchestration completed!")
        print(f"📊 Generated {len(str(results))} specialized responses")
        
        for i, response in enumerate(results, 1):
            print(f"\n   🤖 Agent {i} ({response.name}):")
            preview = response.content[:100] + "..." if len(str(response.content)) > 100 else response.content
            print(f"   📝 {preview}")
            
    except Exception as e:
        print(f"❌ Error in advanced concurrent orchestration: {e}")
        
    finally:
        await runtime3.stop_when_idle()
    
    print("\n🎉 Advanced features demonstration complete!")
    print("💡 Key takeaways:")
    print("   ✅ Agent response callbacks provide real-time monitoring")
    print("   ✅ Proper timeout handling prevents hung operations")
    print("   ✅ Both sequential and concurrent patterns work with advanced features")
    print("   ✅ Semantic Kernel provides robust orchestration infrastructure")

print("✅ Advanced features demo function ready!")


await demonstrate_advanced_features()

✅ Advanced features demo function ready!
🚀 Advanced Orchestration Features Demo
🎭 Specialized agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

👂 1. Enhanced Agent Response Monitoring Demo
----------------------------------------
🎯 Running monitored orchestration...
🔍 Research completed for: Global renewable energy trends with emphasis on solar power adoption (capacity growth, cost trends, policy drivers, technology improvements, major markets, future projections)

[10:53:52.297] 🤖 Agent Response:
    📛 Name: Dr_Research
    📝 Content Length: 1349 characters
    🔤 Content Preview: Renewable-energy headline  
• Solar PV is the engine of global renewables growth—≈65 % of all new renewable capacity in 2023.

Key solar-adoption fact...
    📊 Metadata: {'logprobs': None, 'id': 'chatcmpl-BecIGUad2z5Vxn1yOk0kL4IuNp7V1', 'created': 1749020020, 'system_fingerprint': None, 'usage': CompletionUsage

## 🎓 Congratulations! You're Now an Agent Orchestration Master!

### What You've Accomplished:

✅ **Orchestration Fundamentals**:
- Understanding sequential vs concurrent patterns
- Creating specialized agents with focused expertise
- Building complex multi-agent workflows

✅ **Practical Implementation**:
- Built a Research + Analytics + Content agent team
- Implemented both orchestration patterns
- Compared performance and characteristics

✅ **Advanced Concepts**:
- Learned when to use each pattern
- Explored hybrid orchestration approaches
- Mastered best practices and error handling

### 🚀 Advanced Orchestration Capabilities:

Now you can create:
- 🏭 **Complex Workflows**: Multi-stage processing pipelines
- ⚡ **High-Performance Systems**: Parallel agent processing
- 🎯 **Specialized Teams**: Domain-expert agent collaboration
- 🔄 **Adaptive Orchestration**: Dynamic workflow routing
- 📊 **Intelligent Systems**: Context-aware agent coordination

### 🔮 Next Level Topics:

1. **Dynamic Orchestration**: Agents that decide workflow patterns
2. **Agent Communication**: Direct agent-to-agent messaging
3. **Workflow Optimization**: AI-driven orchestration tuning
4. **Distributed Orchestration**: Cross-service agent coordination
5. **Real-time Orchestration**: Streaming and event-driven patterns

### 💡 Key Takeaways:

- **Sequential orchestration creates quality through careful progression**
- **Concurrent orchestration achieves speed through parallel processing**
- **Agent specialization leads to better results than generalization**
- **Proper error handling and cleanup are essential for production use**
- **Choose orchestration patterns based on your specific requirements**

**Keep building amazing multi-agent systems with Azure AI and Semantic Kernel!** 🌟

---

*Remember: The future of AI is not just single agents, but intelligent teams of specialized agents working together to solve complex problems!*